# جدول المحتويات
- [حول Guidance](../../../../code/01.Introduce)
- [الإعداد](../../../../code/01.Introduce)
- [التوليد غير المقيد](../../../../code/01.Introduce)
- [التحدث باسم Phi 3](../../../../code/01.Introduce)
- [Regex](../../../../code/01.Introduce)
- [اختيار](../../../../code/01.Introduce)
- [سلسلة الأفكار](../../../../code/01.Introduce)
- [توليد JSON](../../../../code/01.Introduce)
- [توليد HTML](../../../../code/01.Introduce)


# عن Guidance

Guidance هي مكتبة مفتوحة المصدر مثبتة بلغة Python للتحكم في مخرجات أي نموذج لغوي (LM). من خلال استدعاء واحد لـ API، يمكنك التعبير (بلغة Python) عن القيود البرمجية الدقيقة التي يجب أن يتبعها النموذج وإنشاء المخرجات المهيكلة بصيغ مثل JSON، Python، HTML، SQL، أو أي هيكل يتطلبه الاستخدام.

تختلف Guidance عن تقنيات التوجيه التقليدية. فهي تفرض القيود عن طريق توجيه النموذج خطوة بخطوة في طبقة الاستدلال، مما ينتج مخرجات ذات جودة أعلى ويقلل التكلفة والوقت بنسبة تصل إلى 30–50% عند استخدامها في سيناريوهات ذات هيكلية عالية.

للتعرف أكثر على Guidance، قم بزيارة [المستودع العام على GitHub](https://github.com/guidance-ai/guidance) أو شاهد [جلسة Guidance Breakout](https://www.youtube.com/watch?v=qXMNPVVlCMs) في Microsoft Build.


# الإعداد
1. قم بتثبيت Guidance باستخدام `pip install guidance --pre`
2. قم بنشر نقطة نهاية Phi 3.5 mini في Azure عن طريق زيارة https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml والنقر على زر "نشر"
3. قم بتخزين مفتاح API الخاص بنقطة النهاية في متغير بيئة يسمى `AZURE_PHI3_KEY` وعنوان URL في متغير بيئة يسمى `AZURE_PHI3_URL`


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# إنشاء غير مقيد  
يمكن إنشاء النصوص دون أي قيود باستخدام الدالة `gen()`. هذا يشبه استخدام النموذج بدون Guidance.  

## تنسيق المحادثة  
مثل العديد من نماذج المحادثة، يتوقع Phi-3 الرسائل بين المستخدم والمساعد بتنسيق محدد. يدعم Guidance قالب محادثة Phi-3 وسيتولى إدارة تنسيق المحادثة نيابةً عنك. لإنشاء أدوار المحادثة، ضع كل جزء من المحادثة في كتلة `with user()` أو `with assistant()`. يمكن استخدام كتلة `with system()` لتعيين رسالة النظام.  


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## توفير الرموز
في السيناريوهات ذات الهيكلية العالية، يمكن لـ Guidance تخطي الرموز وتوليد الرموز الضرورية فقط، مما يحسن الأداء، يزيد الكفاءة ويوفر تكاليف واجهة البرمجة. يتم عرض الرموز المولدة في هذا الدفتر مع خلفية مميزة. الرموز المفروضة تُعرض بدون تمييز وتكلف نفس تكلفة رموز الإدخال، والتي تُقدر بثلث تكلفة رموز الإخراج.

*ملاحظة:* المثال الأول مع التوليد غير المقيد لم يتمكن من فرض أي رموز لأننا لم نقدم أي قيود.


# التحدث عن Phi 3  
بواسطة Guidance، يمكنك بسهولة إدخال النص في استجابات النموذج. يمكن أن يكون هذا مفيدًا إذا كنت ترغب في توجيه مخرجات النموذج في اتجاه معين.


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# تقييد باستخدام التعبيرات النمطية
في المثال السابق، قدم Phi 3 تفسيرات إضافية بعد الإجابة على السؤال بـ `Canberra`. لتقييد إخراج النموذج ليكون كلمة واحدة فقط، يمكن استخدام تعبير نمطي.


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

مع التعبير النمطي، يتم إنشاء الكلمة `كانبيرا` فقط.


# اختيار من بين خيارات متعددة
عندما تكون بعض الخيارات المحتملة معروفة، يمكنك استخدام الدالة `select()` لجعل النموذج يختار من قائمة الخيارات.


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

مع `select()`، تم إنشاء الرمز `Can` فقط. نظرًا لأن `Canberra` هو الخيار الوحيد الذي يمكنه إكمال الاستجابة، تم إجبار الرموز المتبقية.


# سلسلة التفكير
سلسلة التفكير هي تقنية يمكن أن تساعد في تحسين جودة مخرجات النموذج من خلال تشجيعه على معالجة المشكلة خطوة بخطوة. عادةً، للوصول إلى الإجابة النهائية، تكون هناك حاجة إلى عدة مراحل من التوجيه. أولاً، يتم توجيه النموذج للتفكير خطوة بخطوة. ثم يتم توجيه النموذج مرة أخرى لتقديم الإجابة النهائية. مع واجهات برمجة التطبيقات القياسية لاستنتاج المحادثات، يتطلب ذلك مكالمتين API، ويتم احتساب "سلسلة التفكير" التي يولدها النموذج مرتين – مرة كرموز مخرجات عندما يولدها النموذج، ومرة أخرى كرموز إدخال للمكالمة الثانية. مع Guidance، يتم معالجة العملية متعددة الخطوات بالكامل واحتسابها كجزء من مكالمة API واحدة، مما يقلل التكلفة والوقت المستغرق.


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# إنشاء JSON  
يمكن استخدام Guidance لضمان إنشاء JSON متوافق مع مخطط JSON أو نموذج pydantic، مثل مخطط ملف تعريف المستخدم الموضح هنا.  


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## إنشاء صفحات HTML

يمكن استخدام Guidance لإنشاء التعليمات البرمجية والالتزام بالمتطلبات النحوية في لغة البرمجة. في هذا القسم، سنقوم بإنشاء برنامج Guidance صغير لكتابة صفحات ويب HTML بسيطة جدًا.

سنقوم بتقسيم صفحة الويب إلى أقسام أصغر، كل قسم يحتوي على وظيفة Guidance خاصة به. ثم يتم دمج هذه الأقسام في وظيفتنا النهائية لإنشاء صفحة ويب HTML.  
بعد ذلك، سنقوم بتشغيل هذه الوظيفة باستخدام نموذج مدعوم بـ Guidance في Azure AI.

*ملاحظة:* هذا لن يكون مولد HTML متكامل الميزات؛ الهدف هو توضيح كيفية إنشاء مخرجات منظمة تلبي احتياجاتك الفردية.

نبدأ باستيراد ما نحتاجه من Guidance:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

صفحات الويب HTML منظمة للغاية، وسنقوم "بفرض" تلك الأجزاء من الصفحة باستخدام التوجيهات.  
عندما نطلب نصًا بشكل صريح من النموذج، يجب أن نتأكد من أنه لا يتضمن أي شيء يمكن أن يكون علامة - أي يجب أن نستبعد الأحرف '<' و '>'.  


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

يمكننا بعد ذلك استخدام هذه الوظيفة لتوليد نص داخل وسم HTML عشوائي:


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

الآن، دعونا ننشئ عنوان الصفحة.  
كجزء من ذلك، نحتاج إلى إنشاء عنوان للصفحة:


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

يتم ملء جسم صفحة HTML بالعناوين والفقرات.  
يمكننا تعريف وظيفة للقيام بكل منهما:


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

الآن، الوظيفة لتحديد محتوى جسم ملف HTML نفسه.  
تستخدم `select()` مع `recurse=True` لإنشاء عناوين وفقرات متعددة:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

بعد ذلك، نصل إلى الوظيفة التي تُنشئ صفحة HTML كاملة.  
نضيف علامة البداية لـ HTML، ثم نقوم بإنشاء الرأس، ثم الجسم، وبعد ذلك نُضيف علامة النهاية لـ HTML:


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

نحن نوفر غلافًا سهل الاستخدام، والذي سيمكننا من:
- ضبط درجة حرارة التوليد
- التقاط الصفحة المُولدة من كائن النموذج


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

يمكننا بعد ذلك كتابة المخرجات إلى ملف:


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

و [شاهد النتيجة](../../../../code/01.Introduce/sample_page.html).



---

**إخلاء المسؤولية**:  
تم ترجمة هذه الوثيقة باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار الوثيقة الأصلية بلغتها الأصلية المصدر الرسمي. للحصول على معلومات حاسمة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة ناتجة عن استخدام هذه الترجمة.
